In [1]:
import time
start = time.time()

import re
import numpy as np
import pandas as pd
from datetime import timedelta
from unicodedata import normalize, combining

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='luizhemelo')

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

PATH = "../../dados_enem"

## Funções auxiliares

In [2]:
# city location cache
cities = {
    'poxereu':[-15.8290891, -54.3925375, "mato grosso"],
    'brasilia':[-10.3333333, -53.2, "distrito federal"]
}
cols = ['NO_MUNICIPIO_PROVA']

def format_name(name: str):
    """
    Format name to remove punctuation and spaces
    """
    name = re.sub("([^a-zà-üA-ZÀ-Ü0-9])", " ", name)
    nfkd_str = normalize("NFKD", name)
    name = "".join(
        [c for _, c in enumerate(nfkd_str) if not combining(c)]
    ).lower()
    while "  " in name:
        name = name.replace("   ", " ")
        name = name.replace("  ", " ")
    return name

def preprocess_dataset(df: pd.DataFrame, year: int):
    ctt = df['NO_MUNICIPIO_PROVA'].value_counts().index.tolist()
    occ = df['NO_MUNICIPIO_PROVA'].value_counts().values
    df = pd.DataFrame({'Ano':year, 'Num_Candidatos':occ, 'Cidade':ctt})
    
    latitude = []
    longitude = []
    states = []
    
    for i,row in df.iterrows():
        city = format_name(row['Cidade'])
        df.loc[i,'Cidade'] = city
        
        if city not in list(cities.keys()):
            location = geolocator.geocode(str(city+',Brazil'))
            match = geolocator.reverse(location.raw['lat']+","+location.raw['lon'])
            address = match.raw['address']
            state = format_name(address.get('state',''))
            
            cities[city] = [location.raw['lat'], location.raw['lon'], state]
            
            latitude.append(location.raw['lat'])
            longitude.append(location.raw['lon'])
            states.append(state)
        else:
            latitude.append(cities[city][0])
            longitude.append(cities[city][1])
            states.append(cities[city][2])
    
    df['Estado'] = states
    df["id"] = mun_dict[city]
    df['Latitude'] = latitude
    df['Longitude'] = longitude
    
    return df

## Carregamento de códigos de municípios do IBGE

In [3]:
mun_code = pd.read_excel(f"../data/RELATORIO_DTB_BRASIL_MUNICIPIO.xls", header=6)[["Nome_Município", "Código Município Completo"]]
mun_code["Nome_Município"] = mun_code["Nome_Município"].apply(format_name)
mun_code["Código Município Completo"] = mun_code["Código Município Completo"].astype("str")

mun_dict = {}
for i in range(mun_code.shape[0]):
    mun_dict[mun_code["Nome_Município"][i]] = mun_code["Código Município Completo"][i]
mun_dict

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


{'alta floresta d oeste': '1100015',
 'alto alegre dos parecis': '1100379',
 'alto paraiso': '4128625',
 'alvorada d oeste': '1100346',
 'ariquemes': '1100023',
 'buritis': '3109303',
 'cabixi': '1100031',
 'cacaulandia': '1100601',
 'cacoal': '1100049',
 'campo novo de rondonia': '1100700',
 'candeias do jamari': '1100809',
 'castanheiras': '1100908',
 'cerejeiras': '1100056',
 'chupinguaia': '1100924',
 'colorado do oeste': '1100064',
 'corumbiara': '1100072',
 'costa marques': '1100080',
 'cujubim': '1100940',
 'espigao d oeste': '1100098',
 'governador jorge teixeira': '1101005',
 'guajara mirim': '1100106',
 'itapua do oeste': '1101104',
 'jaru': '1100114',
 'ji parana': '1100122',
 'machadinho d oeste': '1100130',
 'ministro andreazza': '1101203',
 'mirante da serra': '1101302',
 'monte negro': '1101401',
 'nova brasilandia d oeste': '1100148',
 'nova mamore': '1100338',
 'nova uniao': '3136603',
 'novo horizonte do oeste': '1100502',
 'ouro preto do oeste': '1100155',
 'parecis'

## Carregamento dos dados e pré-processamento

In [4]:
# 01
enem13 = pd.read_csv(f"{PATH}/microdados_enem_2013/DADOS/MICRODADOS_ENEM_2013.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem13 = preprocess_dataset(enem13, 2013)
enem13

In [ ]:
# 02
enem14 = pd.read_csv(f"{PATH}/microdados_enem_2014/DADOS/MICRODADOS_ENEM_2014.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem14 = preprocess_dataset(enem14, 2014)
enem14

In [ ]:
# 03
enem15 = pd.read_csv(f"{PATH}/microdados_enem_2015/DADOS/MICRODADOS_ENEM_2015.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem15 = preprocess_dataset(enem15, 2015)
enem15

In [ ]:
# 04
enem16 = pd.read_csv(f"{PATH}/microdados_enem_2016/DADOS/MICRODADOS_ENEM_2016.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem16 = preprocess_dataset(enem16, 2016)
enem16

In [ ]:
# 05
enem17 = pd.read_csv(f"{PATH}/microdados_enem_2017/DADOS/MICRODADOS_ENEM_2017.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem17 = preprocess_dataset(enem17, 2017)
enem17

In [ ]:
# 06
enem18 = pd.read_csv(f"{PATH}/microdados_enem_2018/DADOS/MICRODADOS_ENEM_2018.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem18 = preprocess_dataset(enem18, 2018)
enem18

In [ ]:
# 07
enem19 = pd.read_csv(f"{PATH}/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem19 = preprocess_dataset(enem19, 2019)
enem19

In [ ]:
# 08
enem20 = pd.read_csv(f"{PATH}/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem20 = preprocess_dataset(enem20, 2020)
enem20

In [ ]:
# 09
enem21 = pd.read_csv(f"{PATH}/microdados_enem_2021/DADOS/MICRODADOS_ENEM_2021.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem21 = preprocess_dataset(enem21, 2021)
enem21

In [ ]:
# 10
enem22 = pd.read_csv(f"{PATH}/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv", sep=';', encoding = "ISO-8859-1")[cols]
enem22 = preprocess_dataset(enem22, 2022)
enem22

## Finalização da base de dados final

In [ ]:
final = pd.concat([enem13,enem14,enem15,enem16,enem17,enem18,enem19,enem20,enem21,enem22]).reset_index(drop=True)
final[["Latitude", "Longitude"]] = final[["Latitude", "Longitude"]].apply(pd.to_numeric)

final.to_pickle(f"{PATH}/enem_data.pickle")
final.to_csv(f"{PATH}/enem_data.csv")

final.dtypes

In [ ]:
final

In [ ]:
print("Elapsed time (h:mm:ss):",str(timedelta(seconds=(time.time() - start))))